# Capstone Project - The Battle of Neighborhoods (Week 1)

# 1. Introduction

In this section a description of the problem and a discussion of the background are provided.

## 1.1 Background information

Moving from one place to another is something that is done very frequently in our times and can both be very exciting and challenging. There are many reasons for it, e.g. beginning a new job, studying in another city or moving together with your partner. People often want to find a similar place to the one they lived before, especially when they got used to their former neighborhood and felt very comfortable there. Sometimes they also want to change a few things and put more emphasis on different venues nearby, e.g. a park or a certain kind of shop/restaurant. This problem can be tackled using data science methods.

## 1.2 Problem statement

We are exploring neighbourhoods in the city of Munich, Germany, to find a new apartment for Anna that meets her requirements. Anna is 30 years old and at the moment she is living in the borough 'Maxvorstadt' where she feels very comofortable. However, she is only allowed to stay in her current apartment until the end of the year and there are a few things she does not like about it, e.g. that it is very expensive (19.50 €/m²), has no park nearby and that the surrounding area is a little bit too crowded. As a result she is looking for a new apartment in another neighbourhood that is ideally very similar to Maxvorstadt. In addition, Anna would enjoy to have the following venues nearby:

| Venue | Reason | Priority |
| --- | --- | --- |
| Park | Anna likes to take a walk in the park after work. | 1 |
| Tram Station | Anna requires a tram station nearby to get to work easily.  | 2 |
| German Restaurant | Anna often comes home late from work and has no time to cook on her own. She is very much into German food.  | 3 |
| Thai Restaurant | Anna also likes Thai food.  | 4 |
| Cupcake Shop | Anna loves cupcakes and is thinking about opening her own shop.  | 5 |

We will use data science methods to identify the most promissing neighborhoods based on these criteria.

## 1.3 Target audience

Solving this problem can be interesting for everyone who is moving to another city, not exclusively for Anna. The method can be easily transferred to other cities and different people with different needs and preferences. It would also be possible for real estate agents to use similar algorithms to find the perfect home for their customers.

# 2. Data

In this section a description of the data and how it will be used to solve the problem is provided.

## 2.1 Data description

In order to find the most promising neighborhood for Anna the following data is needed:
* A list of existing boroughs in Munich, including additional information like Area in km² and Inhabitants per km². This data is gathered through web scraping from the following webpage: 'https://en.wikipedia.org/wiki/Population_growth_of_Munich#Age_distribution'.
* Geospatial data (latitude, longitude) for each borough. This data is gathered using the Geocoder API based on Open Street Map data and is used as an input for the Foursquare API.
* Average rental prices for all boroughs in Munich. This data is gathered through web scraping from the following webpage: 'https://www.tz.de/leben/wohnen/uebersicht-muenchner-mieten-preise-nach-postleitzahlen-tz-6133643.html'. To work with this data, the postal codes need to be attributed to the corresponding borough via 'https://www.muenchen.de/leben/service/postleitzahlen.html'.
* Information about nearby venues for all boroughs of Munich. This data is gathered using the Foursquare API.

## 2.2 Data usage

We will start by preparing the data from the websites so that we have one overview table that contains all the information (borough, longitude, latitude, rental price, inhabitants per km²) we need. In a next step we peform an exploratory data analysis (using the describe method and a histogram plot) to get insights in the average rental prices for different boroughs in Munich, to which we can compare Anna's current rent and the average rent in her current borough, Maxvorstadt. The data is visualized in a Folium map.

We will prepare a second table using Foursquare data, which contains the top 100 venues within a radius of 1.5 km for each borough. We use one hot encoding to get the 10 most common venues for each borough. Clustering the neighborhood with k-means will limit the boroughs to the ones that are similar to Maxvorstadt (e.g. are in the same cluster).

Using the data from the first table we will limit our further analyis to boroughs where the rent is cheaper than Anna's current rent and which are less crowded (inhabitants per km²) than Maxvorstadt.

Finally we will visualize Anna's favourite venues color coded on the map and count their occurence within the list of top 100 venues of each borough. Combining our results gives us the possibility to recommend the best borough for Anna.

## 2.3 Data preparation

The data is prepared like described below.

### 2.3.1 Import Python libraries

In [242]:
# import libraries
import pandas as pd
import numpy as np
import geocoder # import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import itertools
print('Libraries imported.')

Libraries imported.


### 2.3.2 Get a list of existing boroughs in Munich, Germany

In [243]:
# read Wikipedia data into pd dataframe
df_nb = pd.read_html('https://en.wikipedia.org/wiki/Population_growth_of_Munich#Age_distribution')[16]

In [244]:
# cleanse data and show dataframe
df_nb = df_nb[0:25]
df_nb.rename(columns={"Name":"Borough"}, inplace=True)
df_nb["Longitude"] = np.nan
df_nb["Latitude"] = np.nan
df_nb.head(10)

,Borough,Area in km²,Inhabitant Count,Inhabitants per km²,Longitude,Latitude
0,Allach-Untermenzing,15.45,27730,1795,NaN,NaN
1,Altstadt-Lehel,3.16,18876,5973,NaN,NaN
2,Aubing-Lochhausen-Langwied,34.06,37857,1111,NaN,NaN
3,Au-Haidhausen,4.22,54382,12887,NaN,NaN
4,Berg am Laim,6.31,39009,6182,NaN,NaN
5,Bogenhausen,23.71,75657,3191,NaN,NaN
6,Feldmoching-Hasenbergl,28.71,54245,1889,NaN,NaN
7,Hadern,9.23,44993,4875,NaN,NaN
8,Laim,5.29,50082,9457,NaN,NaN
9,Ludwigsvorstadt-Isarvorstadt,4.39,45736,10418,NaN,NaN


In [245]:
# print number of boroughs
df_nb.shape

(25, 6)

We now have a datset containing the 25 boroughs of Munich. It also includes additional information like the number of inhabitants per km² to judge whether a borough is less crowded than Maxvorstadt.

### 2.3.3 Add coordinates to each borough (using Geocoder API)

In [246]:
# import geocoder
import geocoder
# test geocoder
g = geocoder.osm('Munich')
print(g.x)
print(g.y)

11.5753822
48.1371079


#### Define function to get coordinates

In [247]:
# define function
def geocoding(mnb):
    # initialize variable to None
    latitude = None
    longitude = None
    # loop until getting the coordinates
    while(latitude is None):
        g = geocoder.osm('München ' + mnb)
        latitude = g.x
        longitude = g.y
    df_nb.loc[i, "Longitude"] = latitude
    df_nb.loc[i, "Latitude"] = longitude

#### Loop through boroughs and append coordinates

In [248]:
# loop through neighborhoods and append coordinates to df
l = len(df_nb["Borough"])
for i in range(0, l):
    geocoding(df_nb["Borough"][i])

#### Show resulting dataframe

In [249]:
# show first ten rows
df_nb.head(10)

,Borough,Area in km²,Inhabitant Count,Inhabitants per km²,Longitude,Latitude
0,Allach-Untermenzing,15.45,27730,1795,11.462973,48.195157
1,Altstadt-Lehel,3.16,18876,5973,11.574582,48.137828
2,Aubing-Lochhausen-Langwied,34.06,37857,1111,11.400221,48.165059
3,Au-Haidhausen,4.22,54382,12887,11.590536,48.128753
4,Berg am Laim,6.31,39009,6182,11.633600,48.133971
5,Bogenhausen,23.71,75657,3191,11.633484,48.154782
6,Feldmoching-Hasenbergl,28.71,54245,1889,11.541275,48.213804
7,Hadern,9.23,44993,4875,11.481842,48.118064
8,Laim,5.29,50082,9457,11.503835,48.144352
9,Ludwigsvorstadt-Isarvorstadt,4.39,45736,10418,11.573366,48.130340


### 2.3.4 Add rental price per squaremeter for each borough

#### First step: Get boroughs of Munich and corresponding postal code

In [250]:
# get postal codes from web
url = 'https://www.muenchen.de/leben/service/postleitzahlen.html'
df_munichPoctcode = pd.read_html(url, header=0)[0]
df_munichPoctcode.rename(columns={'Stadtteil':'Borough', 'Postleitzahl':'PostalCode'}, inplace = True)
df_munichPoctcode.head()

,Borough,PostalCode
0,Allach-Untermenzing,"80995, 80997, 80999, 81247, 81249"
1,Altstadt-Lehel,"80331, 80333, 80335, 80336, 80469, 80538, 80539"
2,Au-Haidhausen,"81541, 81543, 81667, 81669, 81671, 81675, 81677"
3,Aubing-Lochhausen-Langwied,"81243, 81245, 81249"
4,Berg am Laim,"81671, 81673, 81735, 81825"


In [251]:
# Save all boroughs in a list
borough_list = df_munichPoctcode['Borough'].to_list()
borough_list = list(itertools.chain.from_iterable(itertools.repeat(x, 9) for x in borough_list))

# Save all postal codes in a list
df_postal = df_munichPoctcode[['PostalCode']].PostalCode.str.split(",",expand=True)
postalcode_list = []

count = 0
# Copy all postal codes in one column under each other
for element in (range(0,len(df_postal))):
    postalcode_list.extend(df_postal.iloc[count])
    count+=1
postalcode_list

# Combine borough and postal code list in new dataframe 
df_postalcodes = pd.DataFrame(list(zip(borough_list, postalcode_list)), columns =['Borough', 'PostalCode']) 
df_postalcodes.head(20)

# Drop columns that contain NaNs
df_postalcodes.dropna(inplace = True)
df_postalcodes.reset_index(drop=True, inplace=True)
df_postalcodes.head(10)

,Borough,PostalCode
0,Allach-Untermenzing,80995
1,Allach-Untermenzing,80997
2,Allach-Untermenzing,80999
3,Allach-Untermenzing,81247
4,Allach-Untermenzing,81249
5,Altstadt-Lehel,80331
6,Altstadt-Lehel,80333
7,Altstadt-Lehel,80335
8,Altstadt-Lehel,80336
9,Altstadt-Lehel,80469


#### Second step: Get rental prices for boroughs and merge data

In [252]:
# get rental prices from web
df_rent1 = pd.read_html('https://www.tz.de/leben/wohnen/uebersicht-muenchner-mieten-preise-nach-postleitzahlen-tz-6133643.html', header=None)[0]
df_rent2 = pd.read_html('https://www.tz.de/leben/wohnen/uebersicht-muenchner-mieten-preise-nach-postleitzahlen-tz-6133643.html', header=None)[1]
df_rent3 = pd.read_html('https://www.tz.de/leben/wohnen/uebersicht-muenchner-mieten-preise-nach-postleitzahlen-tz-6133643.html', header=None)[2]
df_rent = pd.concat([df_rent1, df_rent2, df_rent3])
df_rent = df_rent[1:]
df_rent.rename(columns={0:"PostalCode", 1:"Rental price", 2:"Rental trend", 3:"Buy price", 4:"Buy trend"}, inplace = True)
df_rent.dropna(inplace=True)
df_rent.head(10)

,PostalCode,Rental price,Rental trend,Buy price,Buy trend
2,80995,1410,"1,1%",5000,"6,8%"
3,80997,1325,"-1,9%",5430,"12,4%"
4,80999,1305,"5,2%",5880,"11,4%"
5,81247,1455,"2,5%",6520,"1,4%"
6,81249,1325,"5,6%",5100,"3,4%"
8,80331,2230,"3,5%",k.A.,k.A.
9,80333,1910,"1,9%",9120,"20,8%"
10,80335,1955,"2,9%",8690,"5,8%"
11,80336,1815,"0,0%",8960,"9,0%"
12,80469,206,"4,8%",8370,"0,5%"


In [253]:
# merge dataframes
df_postalcodes["PostalCode"] = df_postalcodes["PostalCode"].astype(int) 
df_rent["PostalCode"] = df_rent["PostalCode"].astype(int) 
df_rent.set_index(['PostalCode'], inplace = True)
df_postalcodes.set_index(['PostalCode'],inplace = True)

In [254]:
# join dataframes
df_rental_prices = df_postalcodes.join(df_rent)
df_rental_prices.reset_index(inplace = True)
df_rental_prices = df_rental_prices[["Borough", "Rental price"]]
df_rental_prices["Rental price"] = df_rental_prices["Rental price"].apply(pd.to_numeric, errors='coerce')
df_rental_prices.dropna(inplace = True)

# group values based on their borough and calculate mean price
df_rental_prices_grouped = df_rental_prices.groupby('Borough').mean()
df_rental_prices_grouped.head(10)

,Rental price
Borough,
Allach-Untermenzing,1370.555556
Altstadt-Lehel,1877.555556
Au-Haidhausen,1708.571429
Aubing-Lochhausen-Langwied,1378.333333
Berg am Laim,1478.750000
Bogenhausen,1729.375000
Feldmoching-Hasenbergl,1401.250000
Hadern,1456.666667
Laim,1521.428571


In [255]:
# cleanse data and add it to merged dataframe
k = len(df_nb)
df_nb["Rent per m²"] = np.nan
for i in range(0,k):
    df_nb.loc[i, "Rent per m²"] = (df_rental_prices_grouped["Rental price"][i]/100).round(2)
df_nb

,Borough,Area in km²,Inhabitant Count,Inhabitants per km²,Longitude,Latitude,Rent per m²
0,Allach-Untermenzing,15.45,27730,1795,11.462973,48.195157,13.71
1,Altstadt-Lehel,3.16,18876,5973,11.574582,48.137828,18.78
2,Aubing-Lochhausen-Langwied,34.06,37857,1111,11.400221,48.165059,17.09
3,Au-Haidhausen,4.22,54382,12887,11.590536,48.128753,13.78
4,Berg am Laim,6.31,39009,6182,11.633600,48.133971,14.79
5,Bogenhausen,23.71,75657,3191,11.633484,48.154782,17.29
6,Feldmoching-Hasenbergl,28.71,54245,1889,11.541275,48.213804,14.01
7,Hadern,9.23,44993,4875,11.481842,48.118064,14.57
8,Laim,5.29,50082,9457,11.503835,48.144352,15.21
9,Ludwigsvorstadt-Isarvorstadt,4.39,45736,10418,11.573366,48.130340,17.84


The dataframe 'df_nb' now contains all the necessary information for each borough, including name, area, inhabitants, longitude, latitude and average rent.

### 2.3.5 Using Foursquare to explore the area around the boroughs

#### Use geopy library to get the latitude and longitude values of Munich:

In [256]:
address = 'Munich'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Munich, Germany are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Munich, Germany are 48.1371079, 11.5753822.


#### Define Foursquare credentials and version

In [257]:
CLIENT_ID = 'BYC11MLYZJFKLNFDVQZL34Y4RONJ5BPR4RWOC0MV00JRUWOM' # your Foursquare ID
CLIENT_SECRET = '21XP2WR2WDQGN4TQY4VFTYSL5CVO53AI45B34CBUA1X1X2JM' # your Foursquare Secret
VERSION = '20201212' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BYC11MLYZJFKLNFDVQZL34Y4RONJ5BPR4RWOC0MV00JRUWOM
CLIENT_SECRET:21XP2WR2WDQGN4TQY4VFTYSL5CVO53AI45B34CBUA1X1X2JM


#### Create a function to find the top 100 venues within 1.5 km radius for each borough

In [258]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the above function on all boroughs

In [259]:
munich_venues = getNearbyVenues(names=df_nb['Borough'],
                                   latitudes=df_nb['Latitude'],
                                   longitudes=df_nb['Longitude']
                                  )

Allach-Untermenzing
Altstadt-Lehel
Aubing-Lochhausen-Langwied
Au-Haidhausen
Berg am Laim
Bogenhausen
Feldmoching-Hasenbergl
Hadern
Laim
Ludwigsvorstadt-Isarvorstadt
Maxvorstadt
Milbertshofen-Am Hart
Moosach
Neuhausen-Nymphenburg
Obergiesing
Pasing-Obermenzing
Ramersdorf-Perlach
Schwabing-Freimann
Schwabing-West
Schwanthalerhöhe
Sendling
Sendling-Westpark
Thalkirchen-Obersendling-Forstenried-Fürstenried-Solln
Trudering-Riem
Untergiesing-Harlaching


In [260]:
print(str(munich_venues.shape[0]) + " venues found.")
munich_venues.head(5)

1845 venues found.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allach-Untermenzing,48.195157,11.462973,Bäckerei Schuhmair,48.197175,11.459016,Bakery
1,Allach-Untermenzing,48.195157,11.462973,Sport Bittl,48.191447,11.466553,Sporting Goods Shop
2,Allach-Untermenzing,48.195157,11.462973,Trattoria Olive,48.189905,11.466970,Trattoria/Osteria
3,Allach-Untermenzing,48.195157,11.462973,Zur Allacher Mühle,48.198411,11.457869,Bavarian Restaurant
4,Allach-Untermenzing,48.195157,11.462973,Würmtalhof,48.188834,11.460680,German Restaurant


The dataframe 'munich_venues' now contains the top 100 venues of all boroughs in Munich.

In [261]:
# show how many vanues were found for each borough
munich_venues.groupby('Neighborhood')["Venue"].count()

Neighborhood
Allach-Untermenzing                                        21
Altstadt-Lehel                                            100
Au-Haidhausen                                             100
Aubing-Lochhausen-Langwied                                 12
Berg am Laim                                               70
Bogenhausen                                                98
Feldmoching-Hasenbergl                                     21
Hadern                                                     41
Laim                                                       91
Ludwigsvorstadt-Isarvorstadt                              100
Maxvorstadt                                               100
Milbertshofen-Am Hart                                     100
Moosach                                                    54
Neuhausen-Nymphenburg                                      99
Obergiesing                                               100
Pasing-Obermenzing                                       

In [262]:
print('There are {} unique venue categories.'.format(len(munich_venues['Venue Category'].unique())))

There are 224 unique venue categories.


## 2.4 Summary of datasets

Further data analysis can be performed on the two data sets resulting from our data preparation: 
* 'df_nb' (borough, area, inhabitants, longitude, latitude, rent)
* 'munich_venues' (top 100 venues in each of the borough within a radius of 1.5 km).